In [1]:
from harp.clock import decode_harp_clock, align_timestamps_to_anchor_points
from open_ephys.analysis import Session
import datetime
from aind_ephys_rig_qc.temporal_alignment import search_harp_line
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import os
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from scipy.io import loadmat
from scipy.stats import zscore

from open_ephys.analysis import Session
from pathlib import Path
import glob

import spikeinterface as si
import xarray as xr
import json
import seaborn as sns
from PyPDF2 import PdfMerger
from aind_ephys_utils import align 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import re
from aind_dynamic_foraging_basic_analysis.licks.lick_analysis import load_nwb, load_data
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
from hdmf_zarr.nwb import NWBZarrIO
from utils.beh_functions import session_dirs, parseSessionID, load_model_dv, makeSessionDF
from utils.ephys_functions import build_time_window_domain, \
                            build_spike_histogram, \
                            build_spike_histogram_overlap, \
                            fitSpikeModelP, \
                            fitSpikeModelG
import pandas as pd
import pickle

%matplotlib inline
%matplotlib widget

In [2]:
session = 'behavior_751004_2024-12-20_13-26-11'
model_name = 'stan_qLearning_5params'
session_dir = session_dirs(session, model_name = model_name)
data_type = 'curated'
curate_time = True

In [3]:
# load behavior data
nwb = load_nwb_from_filename(session_dir['nwb_beh'])
session_df = makeSessionDF(nwb)

In [5]:
model_dv, session_cut = load_model_dv(session, model_name)
trial_df = makeSessionDF(nwb, cut=session_cut)
session_df = pd.merge(trial_df, model_dv, left_index=True, right_index=True, suffixes=('', '_model'))
for column in session_df.columns:
    if '_model' in column:
        session_df.drop(column, axis=1, inplace=True)
session_df.head()
if 'Q_r' in session_df.columns:
    Qchosen = session_df['Q_l'].values
    Qchosen[np.where(session_df['choice']>0)] = session_df.loc[session_df['choice']>0, 'Q_r']
    session_df['Qchosen'] = Qchosen

KeyError: 'beh_nwb_dir'

In [6]:
model_dv, session_cut = load_model_dv(session, model_name)

KeyError: 'nwb_dir'

In [6]:
drift_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session}_opto_drift_tbl.csv')
opto_drift_tbl = pd.read_csv(drift_file)
opto_drift_tbl.reset_index(drop=True, inplace=True)
qm_dir = os.path.join(session_dir['processed_dir'], f'{session}_qm.json')
with open(qm_dir, 'r') as f:
    qm = json.load(f)

In [10]:
unit_id = 40
spike_times_file = os.path.join(session_dir['ephys_processed_dir_curated'], f'spiketimes.pkl')
with open(spike_times_file, 'rb') as f:
    spike_times_all = pickle.load(f)
spike_times_curr = spike_times_all[unit_id]

In [11]:
if unit_id in opto_drift_tbl['unit_id'].values:
    curr_unit_drift = opto_drift_tbl[opto_drift_tbl['unit_id'] == unit_id].to_dict()
curr_unit_drift = {key: list(value.values())[0] for key, value in curr_unit_drift.items()}
ephys_cut_curr = curr_unit_drift['ephys_cut']   
string_list = ephys_cut_curr.replace("nan", "null")
ephys_cut_curr = json.loads(string_list)
ephys_cut_curr = np.array([np.nan if x is None else x for x in ephys_cut_curr])

ephys_cut_curr[np.isnan(ephys_cut_curr)] = np.array(qm['ephys_cut'])[np.isnan(ephys_cut_curr)]
print(ephys_cut_curr)

[5260.36099265 9400.        ]


In [12]:
if curate_time:
    spike_times_curr = spike_times_curr[(spike_times_curr >= ephys_cut_curr[0]) & (spike_times_curr <= ephys_cut_curr[1])]